In [1]:
from monai.transforms import LoadImaged, EnsureChannelFirstd, Compose
from src.transforms import (
    FindCentroid,
    GetFixedROISize,
    CropToCentroidWithSize,
    PadForCrop,
    Save,
    OrthogonalSlices,
    CropToROI,
    GetZoomedROI,
    PadForZoom,
)
from pathlib import Path

def find_corresponding_scan(segmention_path):
    scan_folder = Path(segmention_path).parent.parent / "scans"
    scan_name = Path(segmention_path).name.split(".")[0][:-2] + ".nii.gz"

    return scan_folder / scan_name

pipeline = Compose([
    LoadImaged(keys=["img", "seg"]),
    EnsureChannelFirstd(keys=["img", "seg"]),
])


segmentation_path = r'D:\premium_data\lumc\monotherapy\split_segmentations\PREM_LU_009_0.nii.gz'
scan_path = find_corresponding_scan(segmentation_path)
data = {'img':scan_path, 'seg':segmentation_path}

output = pipeline(data)


In [5]:
select = Compose(
    [
        FindCentroid(),
        GetFixedROISize(10),
        PadForCrop(),
        CropToCentroidWithSize(),
    ]
)

roi = select(output)

In [6]:
roi['centroid']

array([222, 269, 522])

In [9]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer, seed_everything
from src.model import Model
from src.data import DataModule
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

wandb.init()

wandb.config.aggregation_function = 'max'
wandb.config.method = 'crop'
wandb.config.dim = 3
wandb.config.size = 128
wandb.config.roi_size= 50
wandb.config.test_center = 'amphia'
wandb.config.lesion_target = 'lung'
wandb.config.max_batch_size = 12
wandb.config.seed = 0
wandb.config.max_epochs = 100
wandb.config.patience = 20
wandb.config.lr = 5e-4,
wandb.config.min_lr = 1e-6,
wandb.config.t_max = 20,

dm = DataModule(
    r"C:\Users\user\data\dl_radiomics",
    r"C:\Users\user\data\tables\lesion_followup_curated_v5.csv",
    wandb.config
)

model = Model(wandb.config)

seed_everything(wandb.config.seed)

logger = WandbLogger(
    name="hello4",
    project="project_skeleton_on_lung_lesions",
)

checkpoint_callback = ModelCheckpoint(
    monitor='valid_patient_auc',
    mode='max'
)

early_stopping = EarlyStopping(monitor='valid_patient_auc', patience=wandb.config.patience)

trainer = Trainer(
    max_epochs=wandb.config.max_epochs, gpus=1, deterministic=True, fast_dev_run=False, logger=logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[
        early_stopping
    ]
)

trainer.fit(model, dm)

Global seed set to 0


2022-11-01 07:42:57,740 - Global seed set to 0


GPU available: True, used: True


2022-11-01 07:42:57,744 - GPU available: True, used: True


TPU available: None, using: 0 TPU cores


2022-11-01 07:42:57,745 - TPU available: None, using: 0 TPU cores


Loading dataset: 100%|██████████| 168/168 [00:00<?, ?it/s]

2022-11-01 07:42:57,887 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]




  | Name              | Type        | Params
--------------------------------------------------
0 | model             | DenseNet121 | 11.2 M
1 | train_auc         | BinaryAUROC | 0     
2 | val_auc           | BinaryAUROC | 0     
3 | train_patient_auc | BinaryAUROC | 0     
4 | val_patient_auc   | BinaryAUROC | 0     
--------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.975    Total estimated model params size (MB)


2022-11-01 07:42:58,019 - 
  | Name              | Type        | Params
--------------------------------------------------
0 | model             | DenseNet121 | 11.2 M
1 | train_auc         | BinaryAUROC | 0     
2 | val_auc           | BinaryAUROC | 0     
3 | train_patient_auc | BinaryAUROC | 0     
4 | val_patient_auc   | BinaryAUROC | 0     
--------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.975    Total estimated model params size (MB)


In [5]:
from pathlib import Path
import os

r = Path(r'D:\premium_data\maxima\monotherapy\split_segmentations')

for file in r.iterdir():
    os.rename(file, file.parent / file.name.replace('.seg.nrrd',''))